## Get input information
To use Mesh Editing, we need 
 - centers information: .bin or .xyz files
  - Transformations: dual quaternions .txt files 

In [1]:
from scipy.spatial.transform import Rotation as R
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## bin to xyz 

In [2]:
import struct
#file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test\Data\pent\centers\mesh_000res_1000_000.bin'
file_path = 'G:\VS2022Projects/arap-volume-tracking-main\data\Levi\Frame_00vg_512_000.bin'
print(file_path)

def load_bin(file_path):
    with open(file_path, 'rb') as f:
        c = struct.unpack('i', f.read(4))[0]
        
        vectors = np.zeros((c, 3), dtype=np.float32)
        
        for i in range(c):
            x, y, z = struct.unpack('3f', f.read(12))
            vectors[i] = (x, y, z)
    
    return vectors

centers = load_bin(file_path)
print(centers, centers.__len__())

D:\VS2022Projects/arap-volume-tracking-main\data\Levi\Frame_00vg_512_000.bin


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\VS2022Projects/arap-volume-tracking-main\\data\\Levi\\Frame_00vg_512_000.bin'

## xyz to bin

In [2]:
def save_bin(file_path, centers):
    with open(file_path, 'wb') as f:
        c = len(centers)
        f.write(struct.pack('i', c))
        
        for vector in centers:
            f.write(struct.pack('3f', *vector))

new_file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test\Data\pent\centers\mesh_000res_1000_000_out.bin'
save_bin(new_file_path, centers)

## draw centers

In [8]:
def draw_centers(centers, radius, color):
    spheres = []
    sphere_radius = radius 
    
    for point in centers:
        mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
        mesh_sphere.compute_vertex_normals()
        
        mesh_sphere.translate(point)
        
        spheres.append(mesh_sphere)
    
    all_spheres = o3d.geometry.TriangleMesh()
    for sphere in spheres:
        all_spheres += sphere
    
    all_spheres.paint_uniform_color(color) 
    
    o3d.visualization.draw_geometries([all_spheres])

## Apply Transformation

In [5]:
#file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\pent/transformations_1.txt'
file_path = r'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer\inverse_transformations_005.txt'

real_data = []
dual_data = []

with open(file_path, 'r') as file:
    for line in file:
        values = line.strip().split(';')
        
        real_values = tuple(float(value) for value in values[:4]) 
        dual_values = tuple(float(value) for value in values[4:]) 
        
        real_data.append(real_values)
        dual_data.append(dual_values)

def create_matrix_3x3(real):
    rx, ry, rz, rw = real
    
    matrix = np.zeros((3, 3))

    matrix[0, 0] = rw * rw + rx * rx - ry * ry - rz * rz
    matrix[0, 1] = 2 * rx * ry - 2 * rw * rz
    matrix[0, 2] = 2 * rx * rz + 2 * rw * ry
    matrix[1, 0] = 2 * rx * ry + 2 * rw * rz
    matrix[1, 1] = rw * rw - rx * rx + ry * ry - rz * rz
    matrix[1, 2] = 2 * ry * rz - 2 * rw * rx
    matrix[2, 0] = 2 * rx * rz - 2 * rw * ry
    matrix[2, 1] = 2 * ry * rz + 2 * rw * rx
    matrix[2, 2] = rw * rw - rx * rx - ry * ry + rz * rz
    
    return matrix

print(real_data.__len__())

2000


In [6]:
centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/centers/dancer_fr0res_2000_005.xyz'
centers = np.loadtxt(centers_path)
reference_centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/reference_center/reference_centers_aligned.xyz'
reference_centers = np.loadtxt(reference_centers_path)

draw_centers(reference_centers, 0.01, [1.0,0,0])

In [11]:
def quaternion_multiply(quaternion1, quaternion0):
    x0, y0, z0, w0 = quaternion0
    x1, y1, z1, w1 = quaternion1
    return np.array([-x1*x0 - y1*y0 - z1*z0 + w1*w0,
                         x1*w0 + y1*z0 - z1*y0 + w1*x0,
                        -x1*z0 + y1*w0 + z1*x0 + w1*y0,
                         x1*y0 - y1*x0 + z1*w0 + w1*z0], dtype=np.float64)

In [12]:
transformed_centers = []
Transformations = []
for i in range(len(reference_centers)):
    real_values = real_data[i]  
    dual_values = dual_data[i]  
    r = R.from_quat(real_data[i])
    Transformations.append(r.as_matrix())
    point = reference_centers[i]
    x1 = real_values[0]
    y1 = real_values[1]
    z1 = real_values[2]
    w1 = real_values[3]
    x2 = dual_values[0]
    y2 = dual_values[1]
    z2 = dual_values[2]
    w2 = dual_values[3]
    Conjugate = (-x1,-y1,-z1, w1)
    TQ = quaternion_multiply(dual_values, Conjugate)
    Vector = (TQ[1]*2, TQ[2]*2, TQ[3]*2) # not TQ[0]*2, TQ[1]*2, TQ[2]*2, TQ[0] is w 
    transformed_center = Vector +  np.dot(r.as_matrix(), point)
    #print(transformed_center)
    transformed_centers.append(transformed_center)
    
file_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\pent/transformed_centers_0.txt'
np.savetxt(file_path,transformed_centers)

In [13]:
draw_centers(transformed_centers, 0.01, [1.0,0,0])

## Use transformed centers and centers to get indices and transformation dual quaternions, codes below are used for test and evaluation, to get input of each dataset, refer to Get{dataset}input

In [17]:
transformed_centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\pent/transformed_centers_0.txt'

loaded_transformed_centers = np.loadtxt(transformed_centers_path)
print(loaded_transformed_centers)

FileNotFoundError: G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\pent/transformed_centers_0.txt not found.

In [15]:
def get_dual_quaternions(original_centers, transformed_centers):
    moved_indices = []
    for i in range(len(original_centers)):
        if not (np.array_equal(original_centers[i], transformed_centers[i])):
            moved_indices.append(i)
    dual_quaternions = np.zeros((len(original_centers), 8), dtype=np.float32)
    for i in moved_indices:
        original = original_centers[i]
        transformed = loaded_transformed_centers[i]

        rotation_quaternion = R.from_quat([0, 0, 0, 1]) 
    
        translation = transformed - original
        
        rotation_quat = rotation_quaternion.as_quat()
        
        # Dual quaternion: q + ϵ * (0.5 * t * q)
        translation_quat = np.hstack((translation, [0]))
        dual_quat = np.hstack((rotation_quat, 0.5 * translation_quat))
        dual_quaternions[i] = dual_quat
    return moved_indices, dual_quaternions

In [16]:
indices, dual_quaternions = get_dual_quaternions(centers, transformed_centers)

NameError: name 'loaded_transformed_centers' is not defined

In [37]:
#transformed_centers_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Levi/reference_centers_aligned.xyz'
#transformed_centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Levi/centers/Frame_00res_4000_001.xyz'
transformed_centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/reference_centers_aligned.xyz'
loaded_transformed_centers = np.loadtxt(transformed_centers_path)
#print(loaded_transformed_centers)

#centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/centers/Frame_00res_4000_000.xyz'
centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/centers/dancer_fr0res_2000_005.xyz'
loaded_centers = np.loadtxt(centers_path)
#print(loaded_centers)

indices, dual_quaternions = get_dual_quaternions(loaded_centers, loaded_transformed_centers)

In [33]:
#indices_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Levi/indices.txt'
indices_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/indices.txt'
np.savetxt(indices_path, indices, fmt='%d')

In [34]:
dual_quaternions_path = r'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer\transformations.txt'

# Open the file for writing
with open(dual_quaternions_path, 'w') as file:
    for dq in dual_quaternions:
        # Format the dual quaternion into the desired string format
        dq_str = f"{dq[0]};{dq[1]};{dq[2]};{dq[3]};{dq[4]};{dq[5]};{dq[6]};{dq[7]}"
        file.write(dq_str + '\n')

# Optionally, print the first few lines to verify the saved file
with open(dual_quaternions_path, 'r') as file:
    lines = file.readlines()
    print("".join(lines[:5]))  # Print the first 5 lines to verify

0.0;0.0;0.0;1.0;0.0056657250970602036;0.000894415017683059;-0.006727435160428286;0.0
0.0;0.0;0.0;1.0;-0.005258947145193815;-0.0030945499893277884;0.0071906400844454765;0.0
0.0;0.0;0.0;1.0;-0.0027163515333086252;-0.00023604999296367168;-0.0033335499465465546;0.0
0.0;0.0;0.0;1.0;0.015940947458148003;-0.000542099995072931;0.015590755268931389;0.0
0.0;0.0;0.0;1.0;-0.006049539893865585;-0.0007469850243069232;-0.005049725063145161;0.0



In [100]:
mesh_0 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
mesh_1 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0006.obj')
deformed_mesh_0 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_005.obj')
deformed_mesh_1 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_006.obj')
deformed_mesh_2 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_007.obj')
deformed_mesh_3 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_008.obj')
deformed_mesh_4 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/deformed_reference_mesh_009.obj')
fitting_mesh_5 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\reference/fitting_mesh_dancer_5.obj')
mesh_0.compute_vertex_normals()
mesh_1.compute_vertex_normals()
deformed_mesh_0.compute_vertex_normals()
deformed_mesh_1.compute_vertex_normals()
deformed_mesh_2.compute_vertex_normals()
deformed_mesh_3.compute_vertex_normals()
deformed_mesh_4.compute_vertex_normals()
fitting_mesh_5.compute_vertex_normals()
fitting_mesh_5.paint_uniform_color([0.7, 0.7, 0.7])
mesh_1.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([mesh_0])

In [3]:
mesh1 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\arap-volume-tracking-main\data\Dancer/dancer_fr0001.obj')
mesh2 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\arap-volume-tracking-main\data\Dancer/dancer_fr0002.obj')
mesh3 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\arap-volume-tracking-main\data\Dancer/dancer_fr0003.obj')
mesh4 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\arap-volume-tracking-main\data\Dancer/dancer_fr0004.obj')
mesh1.compute_vertex_normals()
mesh2.compute_vertex_normals()
mesh3.compute_vertex_normals()
mesh4.compute_vertex_normals()

o3d.visualization.draw_geometries([mesh1, mesh2, mesh3, mesh4])

In [17]:
deformed_meshes = []
all_meshes = o3d.geometry.TriangleMesh()
for i in range(5,20):
    mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\output/deformed_{i:03}.obj')
    mesh.compute_vertex_normals()
    #print(mesh)
    deformed_meshes.append(mesh)
    all_meshes += mesh
#o3d.visualization.draw_geometries([all_meshes])
print(all_meshes, all_meshes.has_vertex_normals())
reference_pcd = o3d.geometry.PointCloud()
reference_pcd.points = o3d.utility.Vector3dVector(all_meshes.vertices)
reference_pcd.normals = o3d.utility.Vector3dVector(all_meshes.vertex_normals)
print(reference_pcd, reference_pcd.has_normals())

TriangleMesh with 295815 points and 590681 triangles. True
PointCloud with 295815 points. True


In [70]:
load_reference_mesh = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh.obj')
print(load_reference_mesh)
load_reference_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([load_reference_mesh])

TriangleMesh with 20002 points and 40000 triangles.


In [72]:
import open3d as o3d
import numpy as np
spheres = []

# Define the radius of the spheres
sphere_radius = 0.01  # Adjust the radius as needed
changed_sphere = o3d.geometry.TriangleMesh()
# Create a mesh sphere for each point and translate it to the point's location
count = 0
for i in range(loaded_transformed_centers.__len__()):
    point = loaded_transformed_centers[i]
    # Create a sphere mesh
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
    mesh_sphere.compute_vertex_normals()
    
    # Translate the sphere to the point's location
    mesh_sphere.translate(point)
    mesh_sphere.paint_uniform_color([1, 0, 0])  # Light grey color
    # Add the sphere to the list
    spheres.append(mesh_sphere)
print(spheres.__len__(), count)
# Combine all the spheres into one mesh
post_spheres = o3d.geometry.TriangleMesh()
for sphere in spheres:
    post_spheres += sphere

spheres = []

# Define the radius of the spheres
sphere_radius = 0.01  # Adjust the radius as needed
changed_sphere = o3d.geometry.TriangleMesh()
# Create a mesh sphere for each point and translate it to the point's location
count = 0
for i in range(loaded_centers.__len__()):
    point = loaded_centers[i]
    # Create a sphere mesh
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
    mesh_sphere.compute_vertex_normals()
    
    # Translate the sphere to the point's location
    mesh_sphere.translate(point)
    mesh_sphere.paint_uniform_color([0, 1, 0])  # Light grey color
    # Add the sphere to the list
    spheres.append(mesh_sphere)
print(spheres.__len__(), count)
# Combine all the spheres into one mesh
post_spheres1 = o3d.geometry.TriangleMesh()
for sphere in spheres:
    post_spheres1 += sphere
# Optionally, you can set the color of the spheres
#all_spheres.paint_uniform_color([0.7, 0.7, 0.7])  # Light grey color

# Draw the combined mesh
o3d.visualization.draw_geometries([post_spheres, load_reference_mesh])

2000 0
2000 0


In [ ]:
transformed_centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/reference_centers_aligned.xyz'
loaded_transformed_centers = np.loadtxt(transformed_centers_path)
#print(loaded_transformed_centers)

#centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/centers/Frame_00res_4000_000.xyz'
centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/centers/dancer_fr0res_2000_005.xyz'
loaded_centers = np.loadtxt(centers_path)
#print(loaded_centers)

indices, dual_quaternions = get_dual_quaternions(loaded_centers, loaded_transformed_centers)